In [2]:
from datasets import load_from_disk
from collections import Counter

test_ds = load_from_disk('/home/jovyan/CHJ/Mol-LLM_Custom/dataset/train_official/GSAI-ML-LLaDA-8B-Instruct_string+graph_q32_train_512_Truncation_100_sampled')
Counter(test_ds['task'])


Counter({'bace': 100,
         'chebi-20-mol2text': 100,
         'chebi-20-text2mol': 100,
         'forward_reaction_prediction': 100,
         'qm9_homo': 100,
         'qm9_homo_lumo_gap': 100,
         'qm9_lumo': 100,
         'reagent_prediction': 100,
         'retrosynthesis': 100,
         'smol-forward_synthesis': 100,
         'smol-molecule_captioning': 100,
         'smol-molecule_generation': 100,
         'smol-property_prediction-bbbp': 100,
         'smol-property_prediction-clintox': 100,
         'smol-property_prediction-esol': 100,
         'smol-property_prediction-hiv': 100,
         'smol-property_prediction-lipo': 100,
         'smol-property_prediction-sider': 100,
         'smol-retrosynthesis': 100})

In [ ]:
"""
{
    'bace': x,
    'chebi-20-mol2text': x,
    'chebi-20-text2mol': x,
    'forward_reaction_prediction': o,
    'qm9_homo': o,
    'qm9_homo_lumo_gap': o,
    'qm9_lumo': o,
    'reagent_prediction': o,
    'retrosynthesis': o,
    'smol-forward_synthesis': o,
    'smol-molecule_captioning': o,
    'smol-molecule_generation': o,
    'smol-property_prediction-bbbp': o,
    'smol-property_prediction-clintox': o,
    'smol-property_prediction-esol': o,
    'smol-property_prediction-hiv': o,
    'smol-property_prediction-lipo': o,
    'smol-property_prediction-sider': o,
    'smol-retrosynthesis': o})
"""